In [1]:
# Setting PYTHONPATH to donkeycar
# import sys
# sys.path[1] = '/home/ehs/projects/donkeycar/env/lib/python3.6/site-packages'
# print("modified path: %s" % sys.path)

from donkeycar.utils import *
import donkeycar as dk
import pickle
from tensorflow.python import keras
from PIL import Image
import glob
import numpy as np

using donkey v3.1.0 ...


/home/ehs/projects/donkeycar/env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ehs/projects/donkeycar/env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ehs/projects/donkeycar/env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ehs/projects/donkeycar/env/lib/python3.6/site-

In [2]:
model_path = '/home/ehs/projects/mobilelab_donkeycar/models/labmovel4.h5'
model_type = None
cfg = pickle.load( open( "/home/ehs/projects/mobilelab_donkeycar/cfg.pickle", "rb" ) )

In [3]:
kl = dk.utils.get_model_by_type(model_type, cfg)
kl.load(model_path)
kl.compile()

"get_model_by_type" model Type is: linear
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [7]:
import json
import re 

images = glob.glob('/home/ehs/projects/mobilelab_donkeycar/tub/tub_9_19-12-07_v2/*.jpg')
results = []
prefix = 'record_%d.json'
regex = r"/([0-9]+)_"

for image_name in images:
    im =  Image.open(image_name)
    img_arr = np.array(im)
    steering, throttle = kl.run(img_arr)
    
    matches = re.finditer(regex, image_name, re.MULTILINE)
    for matchNum, match in enumerate(matches, start=1):
        number = match.group(1)
        
    json_name =  '/home/ehs/projects/mobilelab_donkeycar/tub/tub_9_19-12-07_v2/' + (prefix % int(number))
#     print(json_name)
    with open(json_name, 'r') as infile:
        data = json.load(infile)
        in_steering = data['user/angle']
        in_throttle = data['user/throttle']
    
    results.append( (image_name, in_steering, in_throttle, steering, throttle) )

In [9]:
# Save dataset as csv
import csv

data = results

with open('data.csv','w') as out:
    csv_out=csv.writer(out)
    csv_out.writerow(['filename', 'steering_in', 'throttle_in','steering_out', 'throttle_out'])
    for row in data:
        csv_out.writerow(row)

In [10]:
# Data analysis
import pandas as pd

data = pd.read_csv('data.csv')

display(data.describe())
display(data.head())

,steering_in,throttle_in,steering_out,throttle_out
count,5638.000000,5638.000000,5638.000000,5638.000000
mean,-0.121890,0.457287,-5.237981,20.567062
std,0.295061,0.204025,28.868257,18.449362
min,-0.919414,0.000000,-152.254760,-40.201645
25%,-0.241026,0.414652,-11.590780,10.931296
50%,-0.000244,0.521123,0.192828,22.191178
75%,-0.000244,0.589927,7.527064,32.349836
max,0.769963,0.876923,134.068730,90.385480


,filename,steering_in,throttle_in,steering_out,throttle_out
0,/home/ehs/projects/mobilelab_donkeycar/tub/tub...,-0.000244,0.549695,-15.809827,22.359932
1,/home/ehs/projects/mobilelab_donkeycar/tub/tub...,-0.000244,0.611233,-0.073938,14.648738
2,/home/ehs/projects/mobilelab_donkeycar/tub/tub...,-0.591697,0.680342,-19.438745,26.955711
3,/home/ehs/projects/mobilelab_donkeycar/tub/tub...,-0.546764,0.348474,-6.139685,11.954758
4,/home/ehs/projects/mobilelab_donkeycar/tub/tub...,-0.000244,0.000000,-52.271275,-2.218138


In [15]:
# Showing data correlation between inputs and outputs
data.corr()

,steering_in,throttle_in,steering_out,throttle_out
steering_in,1.000000,-0.081870,0.814038,-0.116375
throttle_in,-0.081870,1.000000,-0.102525,0.817369
steering_out,0.814038,-0.102525,1.000000,-0.199761
throttle_out,-0.116375,0.817369,-0.199761,1.000000
